## Following the guide for BERT as practice
https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [84]:
suc3 = pd.read_pickle("../data/sucFrame")
suc3.head(10)

,text_id,text_lix,text_nk,text_ovix,text_swefn,text_index,sentence__geocontext,sentence_id,sentence_index,word_blingbring,...,word_text,word_ex,word_name,word_subtype,word_type,word_index,word_tag,word_sentiment,word_sentimentclass,word__overlap
0,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,NaN,...,I,NaN,NaN,NaN,NaN,1,w,NaN,NaN,NaN
1,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,NaN,...,sin,NaN,NaN,NaN,NaN,2,w,NaN,NaN,NaN
2,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,NaN,...,första,NaN,NaN,NaN,NaN,3,w,NaN,NaN,NaN
3,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,|gengäld|gensträvighet|hinder|motstånd|motverk...,...,reaktion,NaN,NaN,NaN,NaN,4,w,-0.247931,neutral,NaN
4,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,NaN,...,på,NaN,NaN,NaN,NaN,5,w,NaN,NaN,NaN
5,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,NaN,...,Sovjetledarens,NaN,NaN,NaN,NaN,6,w,NaN,NaN,NaN
6,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,|anvisning|befallning|bestraffning|betänklighe...,...,varningar,NaN,NaN,NaN,NaN,7,w,-0.5145,negative,NaN
7,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,|bekräftelse|svar|,...,deklarerade,NaN,NaN,NaN,NaN,8,w,NaN,NaN,NaN
8,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,NaN,...,NaN,ENAMEX,Litauens,PPL,LOC,9,ne,NaN,NaN,NaN
9,aa01c,50.84,1.58,76.88,|Abandonment:95.654|Destroying:87.097|Relation...,1,NaN,e24e30c0-e24d3ca9,1,|ledare|myndighet|överordnad|,...,president,NaN,NaN,NaN,NaN,10,w,-0.0656,neutral,NaN


In [87]:
#Grab the categories the example use
data = suc3[['sentence_index', 'word_text', 'word_pos', 'word_type']]

# The example uses O instaed of Nan, so we follow them
data[['word_type']] = data[['word_type']].replace(np.nan,'O')
data.head(10)

/home/vic/git/thesis/.thesis/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,sentence_index,word_text,word_pos,word_type
0,1,I,PP,O
1,1,sin,PS,O
2,1,första,RO,O
3,1,reaktion,NN,O
4,1,på,PP,O
5,1,Sovjetledarens,NN,O
6,1,varningar,NN,O
7,1,deklarerade,VB,O
8,1,NaN,NaN,LOC
9,1,president,NN,O


In [78]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word_text"].values.tolist(),
                                                           s["word_pos"].values.tolist(),
                                                           s["word_type"].values.tolist())]
        self.grouped = self.data.groupby("sentence_index").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [79]:
getter = SentenceGetter(data)


In [80]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

TypeError: sequence item 8: expected str instance, float found

In [82]:
data.iloc[8]

sentence_index      1
word_text         NaN
word_pos          NaN
word_type         LOC
Name: 8, dtype: object